In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os
import sys
from weight_train_trip2 import *
from weight_train_trip_ev import *
from sklearn.preprocessing import LabelEncoder

In [2]:
data_spot = pd.read_excel('data_spot_5_fold.xlsx')

In [3]:
new_id = pd.DataFrame(columns=["Spot","new_id"])
new_id["Spot"] = data_spot.Spot.unique()
new_id["new_id"] = range(data_spot.Spot.nunique())

In [4]:
spot = pd.read_excel('../최종DB관리/Spot_VSS/2019.02.13.최종_통합DB2.xlsx')

In [5]:
new_id_spot = new_id.merge(spot,on="Spot",how="left")

In [6]:
g = tf.Graph()

In [7]:
class args():
    is_training = True
    retraining = True
    layers = 1
    rnn_size = [300,4000] # , 20, 5 ,50
    n_epochs = 10
    batch_size = 50
    dropout_p_hidden= 1.0
    learning_rate = 0.0005
    decay = 0.96
    decay_steps = 5e3
    init_as_normal = False
    reset_after_session = True
    session_key = 'Session_id'
    attribute_key =['City_encoder','new_id'] #   ,'Tag_encoder'   ,"city_sub_encoder",
    item_key = 'Spot'
    time_key = 'Session_num'
    grad_cap = 0
    attribute = [new_id_spot.City_encoder.nunique(),len(new_id_spot) ]# ,spot.Tag_encoder.nunique()  ,spot.city_sub_encoder.nunique(),
    model_name = ["id","City"]
    max_ckpt_keep = 100
    test_model_train = [8,13] # [13,13,12,12,11]
    test_model = 0
    sigma=0
    checkpoint_dir_train = ['City_encoder_0','new_id_0'] 
    checkpoint_dir = "weight_train2_0"
    graph = g
    loss = 'top1'
    final_act = 'linear'
    hidden_act = 'tanh'
    item_table = new_id_spot

In [8]:
tf.reset_default_graph()

In [9]:
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.Session(graph=g,config=config)
arg = args()
models = GRU4Rec(sess,arg)

INFO:tensorflow:Restoring parameters from City_encoder_0/attribute_model_City_encoder-8
완료
이거는?
INFO:tensorflow:Restoring parameters from new_id_0/attribute_model-13
완료


In [10]:
train = data_spot[data_spot["k"]!=0]
test = data_spot[data_spot["k"]==0]

In [11]:
models.fit(train)

fitting model....
[    0     8    32 ... 22839 22846 22850]
Epoch 0	Step 1	lr: 0.000500	loss: 1.283171 weight:[0.5 0.5]
Epoch 0 	 loss: 1.282229 : 
[    0     8    32 ... 22839 22846 22850]
Epoch 1 	 loss: 1.282771 : 
[    0     8    32 ... 22839 22846 22850]
Epoch 2 	 loss: 1.281681 : 
[    0     8    32 ... 22839 22846 22850]
Epoch 3 	 loss: 1.281004 : 
[    0     8    32 ... 22839 22846 22850]
Epoch 4 	 loss: 1.279377 : 
[    0     8    32 ... 22839 22846 22850]
Epoch 5 	 loss: 1.277705 : 
[    0     8    32 ... 22839 22846 22850]
Epoch 6 	 loss: 1.276258 : 
[    0     8    32 ... 22839 22846 22850]
Epoch 7 	 loss: 1.274227 : 
[    0     8    32 ... 22839 22846 22850]
Epoch 8 	 loss: 1.273008 : 
[    0     8    32 ... 22839 22846 22850]
Epoch 9 	 loss: 1.271221 : 


In [12]:
 
g1= tf.Graph()

class args():
    is_training = False
    retraining = True
    layers = 1
    rnn_size = [300,4000] # , 20, 5 ,50
    n_epochs = 10
    batch_size = 50
    dropout_p_hidden= 1.0
    learning_rate = 0.00001
    decay = 0.96
    decay_steps = 5e3
    init_as_normal = False
    reset_after_session = True
    session_key = 'Session_id'
    attribute_key =['City_encoder','new_id'] #   ,'Tag_encoder'   ,"city_sub_encoder",
    item_key = 'Spot'
    time_key = 'Session_num'
    grad_cap = 0
    attribute = [new_id_spot.City_encoder.nunique(),len(new_id_spot) ]# ,spot.Tag_encoder.nunique()  ,spot.city_sub_encoder.nunique(),
    model_name = ["id","City"]
    max_ckpt_keep = 100
    test_model_train = [8,11] # [13,13,12,12,11]
    test_model = 0
    sigma=0
    checkpoint_dir_train =  ['City_encoder_0','new_id_0'] 
    checkpoint_dir = "weight_train2_0"
    graph = g1
    loss = 'top1'
    final_act = 'linear'
    hidden_act = 'tanh'
    item_table = new_id_spot

In [13]:
tf.reset_default_graph()
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.Session(graph=g1,config=config)
arg = args()
models = GRU4Rec(sess,arg)

INFO:tensorflow:Restoring parameters from City_encoder_0/attribute_model_City_encoder-8
완료
이거는?
INFO:tensorflow:Restoring parameters from new_id_0/attribute_model-11
완료
INFO:tensorflow:Restoring parameters from weight_train2_0/weight-model--0


In [12]:
l_old=evaluate_sessions_batch(models,test,cut_off=20, session_key = 'Session_id', item_key="Spot",time_key= 'Session_num',batch_size=50)

D:\2019.01.04~여행앱개발\modeling\weight_train_trip_ev.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  test_data.sort_values([session_key,time_key],inplace=True)


In [14]:
result = pd.DataFrame(columns=["k","epoch","top","recall","mrr","unique_lst"])

In [15]:

k= 0
for i in range(0,10):
    tf.reset_default_graph()
    g1 = tf.Graph()
    arg = args()
    arg.graph =g1
    config = tf.ConfigProto()
    config.gpu_options.allow_growth=True
    sess = tf.Session(graph=g1,config=config)
    arg.test_model = i
    models = GRU4Rec(sess,arg)
    l_old=evaluate_sessions_batch(models,test, session_key= 'Session_id', item_key='Spot', time_key= 'Session_num',batch_size=50)
    #print(l_old[:2])
    #print(len(set(np.array(l_old[2].Rec_lst.tolist()).reshape(-1))))
    for idx,tt in enumerate([5,10,15,20],1):
        mini=[k,i,tt,l_old[0][idx],l_old[1][idx],len(set(np.array(l_old[2].Rec_lst.apply(lambda x:x[:tt]).tolist()).reshape(-1)))]
        result_mini = pd.Series(mini,index=["k","epoch","top","recall","mrr","unique_lst"])
        result=result.append(result_mini,ignore_index=True)
        print(mini)

INFO:tensorflow:Restoring parameters from City_encoder_0/attribute_model_City_encoder-8
완료
이거는?
INFO:tensorflow:Restoring parameters from new_id_0/attribute_model-11
완료
INFO:tensorflow:Restoring parameters from weight_train2_0/weight-model--0


D:\2019.01.04~여행앱개발\modeling\weight_train_trip_ev.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  test_data.sort_values([session_key,time_key],inplace=True)


[0, 0, 5, 0.43154574132492113, 0.27904311251314395, 332]
[0, 0, 10, 0.5310199789695058, 0.2924408725986012, 573]
[0, 0, 15, 0.5806519453207151, 0.29639472456507143, 741]
[0, 0, 20, 0.6143007360672976, 0.29830286210754203, 850]
INFO:tensorflow:Restoring parameters from City_encoder_0/attribute_model_City_encoder-8
완료
이거는?
INFO:tensorflow:Restoring parameters from new_id_0/attribute_model-11
완료
INFO:tensorflow:Restoring parameters from weight_train2_0/weight-model--1
[0, 1, 5, 0.43154574132492113, 0.2790606379249911, 333]
[0, 1, 10, 0.531440588853838, 0.292531003288101, 575]
[0, 1, 15, 0.582124079915878, 0.2965625588338522, 743]
[0, 1, 20, 0.614931650893796, 0.2984163368532599, 851]
INFO:tensorflow:Restoring parameters from City_encoder_0/attribute_model_City_encoder-8
완료
이거는?
INFO:tensorflow:Restoring parameters from new_id_0/attribute_model-11
완료
INFO:tensorflow:Restoring parameters from weight_train2_0/weight-model--2
[0, 2, 5, 0.43154574132492113, 0.2792113564668769, 331]
[0, 2, 10, 

In [20]:
tf.reset_default_graph()

In [8]:
result = pd.DataFrame(columns=["k","epoch","top","recall","mrr","unique_lst"])
ch_lst = [13,13,12,12,11]
for k in range(5):
    tf.reset_default_graph()
    g = tf.Graph()
    arg = args()
    train = data_spot[data_spot["k"]!=k]
    test = data_spot[data_spot["k"]==k]
    arg.graph = g
    config = tf.ConfigProto()
    config.gpu_options.allow_growth=True
    sess = tf.Session(graph=g,config=config)
    arg.is_training = True
    arg.checkpoint_dir = "weight_train_" + str(k)
    arg.checkpoint_dir_train = ['City_encoder_'+str(k),'new_id_0_'+str(k)] 
    arg.dropout_p_hidden = 1.0
    arg.test_model_train = [8,ch_lst[k]]
    
    models = GRU4Rec(sess,arg)
    models.fit(train)
    
    
    # test#
    arg.is_training = False
    arg.dropout_p_hidden = 1.0
    for i in range(0,10):
        tf.reset_default_graph()
        config = tf.ConfigProto()
        config.gpu_options.allow_growth=True
        sess = tf.Session(config=config)
        arg.test_model = i
        models = GRU4Rec(sess,arg)
        l_old=evaluate_sessions_batch(models,test, session_key= 'Session_id', item_key='Spot', time_key= 'Session_num',batch_size=50)
        #print(l_old[:2])
        #print(len(set(np.array(l_old[2].Rec_lst.tolist()).reshape(-1))))
        for idx,tt in enumerate([5,10,15,20],1):
            mini=[k,i,tt,l_old[0][idx],l_old[1][idx],len(set(np.array(l_old[2].Rec_lst.apply(lambda x:x[:tt]).tolist()).reshape(-1)))]
            result_mini = pd.Series(mini,index=["k","epoch","top","recall","mrr","unique_lst"])
            result=result.append(result_mini,ignore_index=True)
            print(mini)

INFO:tensorflow:Restoring parameters from City_encoder_0/attribute_model_City_encoder-8
완료
이거는?
완료
fitting model....
[    0     8    32 ... 22839 22846 22850]


FailedPreconditionError: Attempting to use uninitialized value gru_layer/softmax_w0
	 [[Node: gru_layer/softmax_w0/read = Identity[T=DT_FLOAT, _class=["loc:@gru_layer/softmax_w0"], _device="/job:localhost/replica:0/task:0/device:GPU:0"](gru_layer/softmax_w0)]]
	 [[Node: gru_layer/BatchNorm/Reshape_1/_9 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_100_gru_layer/BatchNorm/Reshape_1", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'gru_layer/softmax_w0/read', defined at:
  File "C:\Users\LeeWonSeok\Anaconda3\envs\for_tensor\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\LeeWonSeok\Anaconda3\envs\for_tensor\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\LeeWonSeok\Anaconda3\envs\for_tensor\lib\site-packages\ipyker

In [121]:
result 
# 0번 7 epoch
# 1번 9 epoch
# 2번 6 epoch
# 3번 0 epoch
# 4번 0 epoch
result.to_excel('rnn_model_5_fold_result_ID_City(Parellel).xlsx')

In [98]:
# 0번 1 epoch
# 1번 0 epoch
# 2번 0 epoch
# 3번 0 epoch
# 4번 0 epoch
result.to_excel('rnn_model_5_fold_result_ID_Tag_City(Parellel).xlsx')